In [3]:
import pandas as pd
import numpy as np
import re
import Functionalitati

In [2]:
def convert_money(value):
    value = value.replace(",", " ")
    money = int("".join(value.split(" ")[:-1]))
    if value.split(" ")[-1] == "RON":
        money = round(money / 5)
    return money

In [3]:
def convert_area(value):
    value = value.replace(",", ".")
    area = float("".join(value.split(" ")[0]))
    return area

In [4]:
def number_of_floors(value):
    new_value = None
    if value in [np.nan]:
        new_value = np.nan
    elif "/" in value:
        new_value = value.split("/")[1]
    else:
        new_value = np.nan
    return new_value

In [5]:
def apartment_floor(value):
    new_value = None
    if value in [np.nan]:
        new_value = value
    else:
        new_value = re.sub('[<>!,.?*]', "", value.split("/")[0])
        new_value = new_value.strip()
    return new_value

In [6]:
def get_year(value, text):
    new_value = value
    if value in [np.nan]:
        new_values = re.findall("[0-2][0-9][0-9][0-9]", text)
        if new_values == []:
            new_value = np.nan
        else:
            for val in new_values:
                if int(val) < 1900:
                    continue
                else:
                    new_value = val
                    break
    return new_value
    

In [7]:
def parcare_or_garaj(value, text):
    new_value = value
    if text not in [np.nan]:
        text = text.lower()
        words_list = ["garaj", "parcare"]
        if value in [np.nan]:
            for word in words_list:
                if word in text:
                    new_value = word
                    break
    return new_value
            

In [8]:
def building_floors(value, text):
    new_value = value
    flag = 0
    forms = ["\s[0-9][0-9]/[0-9][0-9]\s", "\s[0-9]/[0-9][0-9]\s", "\s[0-9]/[0-9]\s", "parter/[0-9][0-9]", "parter/[0-9]", "demisol/[0-9][0-9]",
             "demisol/[0-9]", "mansardă/[0-9][0-9]", "mansardă/[0-9]", "mansarda/[0-9][0-9]", "mansarda/[0-9]"]
    forms_2 = ['[0-9][0-9] etaje', '[0-9] etaje', '[0-9][0-9] nivele', '[0-9] nivele']
    forms_3 = ['etajul\s[0-9][0-9]/[0-9][0-9]', 'etajul\s[0-9]/[0-9][0-9]', "etajul\s[0-9]/[0-9]", 'etajul\s[0-9][0-9]\sdin\s[0-9][0-9]',
               'etajul\s[0-9]\sdin\s[0-9][0-9]', "etajul\s[0-9]\sdin\s[0-9]", "etaj\s[0-9][0-9]/[0-9][0-9]", "etaj\s[0-9]/[0-9][0-9]",
               "etaj\s[0-9]/[0-9]", 'etaj\s[0-9][0-9]\sdin\s[0-9][0-9]', 'etaj\s[0-9]\sdin\s[0-9][0-9]', "etaj\s[0-9]\sdin\s[0-9]"]
    forms_4 = ['etajul\s[0-9][0-9]', 'etajul\s[0-9]']
    if text not in [np.nan]:
        text = text.lower()
    if (value in [np.nan]) or ("/" not in value):
        flag = 0
        for form in forms:
            new_values = re.findall(form, text)
            if len(new_values) > 0:
                new_value = new_values[0]
                flag = 1
                new_value = new_value.strip()
                break
        if flag == 0:
            for form in forms_3:
                new_values = re.findall(form, text)
                if len(new_values) > 0:
                    new_value = new_values[0].replace(' din ', '/')
                    new_value = new_value.split(" ")[1]
                    new_value = new_value.strip()
                    break
    if ( new_value not in [np.nan]) and ("/" not in new_value):
        flag = 0
        for form in forms_2:
            new_values = re.findall(form, text)
            if len(new_values) > 0:
                flag = 1
                new_value = value + "/" + new_values[0].split(" ")[0]
                new_value = new_value.strip()
                break
    if (flag == 0) and (new_value in [np.nan]):
        for form in forms_4:
            new_values = re.findall(form, text)
            if len(new_values) > 0:
                new_value = new_values[0].split(" ")[1]
                new_value = new_value.strip()
                break
                
    return new_value 

In [9]:
def set_max_floor(current_floor, max_floor, mode_mansarda):
    new_max = max_floor
    if (current_floor == 'mansardă') and (max_floor ==0):
        new_max = mode_mansarda
    elif (current_floor == "parter") or (current_floor == "demisol") or (current_floor == 'mansardă'):
        pass
    elif int(current_floor) > int(max_floor):
        new_max = current_floor
    return new_max   

In [10]:
def dangerous_building(year, max_floor):
    status = None
    year = int(year)
    max_floor = int(max_floor)
    if (year <= 1977) and (max_floor > 5):
        status = "Cade"
    elif (year <= 1977) or (max_floor > 5):
        status = "Te simti norocos?"
    else:
        status = "Sigur"
    return status    

In [11]:
def city_zone(zone_value):
    zone_value = zone_value.split(",")
    if "iasi" in zone_value[-2].lower():
        if len(zone_value) == 4:
            zone = zone_value[1]
        else:
            zone = zone_value[0]
    else:
        if len(zone_value) >= 3:
            zone = zone_value[1]
        else:
            zone = zone_value[0]
    return zone.strip()

In [12]:
def year_mode(zone_value, year_value, df):
    if year_value == 0:
        zone_mode = df[df['Zona Oras'] == zone_value]['Anul construcției'].mode().to_list()[0]
    else:
        zone_mode = year_value
    return zone_mode

In [13]:
vanzare_df = pd.read_excel("apartamente_de_vanzare.xlsx")
inchiriere_df = pd.read_excel("apartamente_de_inchiriat.xlsx")

In [14]:
vanzare_df.replace("Cere informații", np.nan, inplace=True)
vanzare_df.replace("fără informații", np.nan, inplace=True)

In [15]:
inchiriere_df.replace("Cere informații", np.nan, inplace=True)
inchiriere_df.replace("fără informații", np.nan, inplace=True)

In [16]:
vanzare_df['Pret'] = vanzare_df['Pret'].apply(convert_money)
inchiriere_df['Pret'] = inchiriere_df['Pret'].apply(convert_money)

In [17]:
vanzare_df['Suprafață'] = vanzare_df['Suprafață'].apply(convert_area)
inchiriere_df['Suprafață'] = inchiriere_df['Suprafață'].apply(convert_area)

In [18]:
vanzare_df['Etaj'] = vanzare_df.apply(lambda df: building_floors(df["Etaj"], df["Descriere"]), axis=1)
inchiriere_df['Etaj'] = inchiriere_df.apply(lambda df: building_floors(df["Etaj"], df["Descriere"]), axis=1)

In [19]:
vanzare_df['Etaj apartament'] = vanzare_df['Etaj'].apply(apartment_floor)
inchiriere_df['Etaj apartament'] = inchiriere_df['Etaj'].apply(apartment_floor)

In [20]:
vanzare_df['Numar Etaj Bloc'] = vanzare_df['Etaj'].apply(number_of_floors)
inchiriere_df['Numar Etaj Bloc'] = inchiriere_df['Etaj'].apply(number_of_floors)

In [21]:
vanzare_df["Anul construcției"] = vanzare_df.apply(lambda df: get_year(df["Anul construcției"], df["Descriere"]), axis=1)
inchiriere_df["Anul construcției"] = inchiriere_df.apply(lambda df: get_year(df["Anul construcției"], df["Descriere"]), axis=1)

In [22]:
vanzare_df['Garaj/loc de parcare'] = vanzare_df.apply(lambda df: parcare_or_garaj(df['Garaj/loc de parcare'], df["Descriere"]), axis=1)
inchiriere_df['Garaj/loc de parcare'] = inchiriere_df.apply(lambda df: parcare_or_garaj(df['Garaj/loc de parcare'], df["Descriere"]), axis=1)

In [23]:
vanzare_df['Garaj/loc de parcare'].replace(np.nan, "Fara loc", inplace=True)
inchiriere_df['Garaj/loc de parcare'].replace(np.nan, "Fara loc", inplace=True)

In [24]:
new_vanzare_df = vanzare_df.loc[:,("Zona", "Pret", 'Suprafață', 'Numărul de camere', 'Anul construcției',
                                   'Etaj apartament', 'Numar Etaj Bloc', 'Garaj/loc de parcare')]
new_inchiriere_df = inchiriere_df.loc[:, ("Zona", "Pret", 'Suprafață', 'Numărul de camere', 'Anul construcției',
                                          'Etaj apartament', 'Numar Etaj Bloc', 'Garaj/loc de parcare')]

In [25]:
new_vanzare_df['Anul construcției'].replace(np.nan, 0, inplace=True)
new_vanzare_df['Anul construcției'] = new_vanzare_df['Anul construcției'].astype('int')
new_inchiriere_df["Anul construcției"].replace(np.nan, 0, inplace=True)
new_inchiriere_df["Anul construcției"] = new_inchiriere_df["Anul construcției"].astype("int")

In [26]:
new_vanzare_df['Numar Etaj Bloc'].replace(np.nan, 0, inplace=True)
index_to_drop = new_vanzare_df.index[new_vanzare_df['Numar Etaj Bloc'] == "24"].to_list()
new_vanzare_df.drop(index_to_drop, inplace=True)

In [27]:
new_inchiriere_df['Numar Etaj Bloc'].replace(np.nan, 0, inplace=True)
index_to_drop = new_inchiriere_df.index[new_inchiriere_df['Numar Etaj Bloc'] == "24"].to_list()
new_inchiriere_df.drop(index_to_drop, inplace=True)

In [28]:
new_vanzare_df['Etaj apartament'].replace(np.nan, 0, inplace=True)
index_to_drop = new_vanzare_df[new_vanzare_df['Etaj apartament'] == 0].index
new_vanzare_df.drop(index_to_drop, inplace=True)

In [29]:
new_inchiriere_df['Etaj apartament'].replace(np.nan, 0, inplace=True)
index_to_drop = new_inchiriere_df[new_inchiriere_df['Etaj apartament'] == 0].index
new_inchiriere_df.drop(index_to_drop, inplace=True)

In [30]:
mode_mansarda = int(new_vanzare_df[new_vanzare_df['Etaj apartament'] == 'mansardă']['Numar Etaj Bloc'].mode().to_list()[0])
new_vanzare_df['Numar Etaj Bloc'] = new_vanzare_df.apply(lambda df:
                                                         set_max_floor(df['Etaj apartament'], df['Numar Etaj Bloc'], mode_mansarda), axis=1)

In [31]:
mode_mansarda = int(new_inchiriere_df[new_inchiriere_df['Etaj apartament'] == 'mansardă']['Numar Etaj Bloc'].mode().to_list()[0])
new_inchiriere_df['Numar Etaj Bloc'] = new_inchiriere_df.apply(lambda df:
                                                         set_max_floor(df['Etaj apartament'], df['Numar Etaj Bloc'], mode_mansarda), axis=1)

In [32]:
new_vanzare_df['Zona Oras'] = new_vanzare_df['Zona'].apply(city_zone)
new_inchiriere_df['Zona Oras'] = new_inchiriere_df['Zona'].apply(city_zone)

In [33]:
new_vanzare_df["Anul construcției"] = new_vanzare_df.apply(lambda df:
                                                              year_mode(df['Zona Oras'], df["Anul construcției"], new_vanzare_df), axis=1)
new_inchiriere_df["Anul construcției"] = new_inchiriere_df.apply(lambda df:
                                                              year_mode(df['Zona Oras'], df["Anul construcției"], new_inchiriere_df), axis=1)

In [34]:
new_vanzare_df['Cutremur'] = new_vanzare_df.apply(lambda df:
                                                  dangerous_building(df['Anul construcției'], df['Numar Etaj Bloc']), axis=1)
new_inchiriere_df['Cutremur'] = new_inchiriere_df.apply(lambda df:
                                                  dangerous_building(df['Anul construcției'], df['Numar Etaj Bloc']), axis=1)


In [35]:
new_vanzare_df

,Zona,Pret,Suprafață,Numărul de camere,Anul construcției,Etaj apartament,Numar Etaj Bloc,Garaj/loc de parcare,Zona Oras,Cutremur
0,"Podu Ros, Iasi, Iasi",168000,66.18,2,2015,4,4,Fara loc,Podu Ros,Sigur
1,"Dacia, Iasi, Iasi",128000,81.00,3,1989,8,9,Fara loc,Dacia,Te simti norocos?
2,"Podu de Fier, Iasi, Iasi",396000,230.00,5,2001,3,3,parcare,Podu de Fier,Sigur
3,"Agronomie, Iasi, Iasi",102405,68.27,3,2023,2,2,Fara loc,Agronomie,Sigur
4,"Soseaua Nicolina, CUG, Iasi, Iasi",120000,78.00,3,2023,6,10,Fara loc,CUG,Te simti norocos?
...,...,...,...,...,...,...,...,...,...,...
4484,"Bucium, Iasi, Iasi",55000,52.00,2,2021,4,7,parcare,Bucium,Te simti norocos?
4485,"Soseaua Nicolina, Nicolina 1, Iasi, Iasi",98000,96.00,4,1988,7,7,Fara loc,Nicolina 1,Te simti norocos?
4486,"Strada Baltagului, Agronomie, Iasi, Iasi",86850,64.50,2,2022,2,10,parcare,Agronomie,Te simti norocos?
4487,"Strada Baltagului, Agronomie, Iasi, Iasi",115075,88.00,3,2022,3,10,parcare,Agronomie,Te simti norocos?
